In [1]:
# Start corenlp server by running:
#    giveme5w1h-corenlp
# corenlp does not work on new versions of Java. Install Java 8 in a conda environment with:
#    conda install -c cidermole jdk8
# Must add this caching file manually:
#    mkdir -p ~/anaconda3/envs/CONDA_ENV_NAME/lib/python3.7/site-packages/Giveme5W1H/examples/caches
#    cd ~/anaconda3/envs/CONDA_ENV_NAME/lib/python3.7/site-packages/Giveme5W1H/examples/caches
#    wget https://github.com/fhamborg/Giveme5W1H/blob/master/Giveme5W1H/examples/caches/_Nominatim.prickle?raw=true
#    mv _Nominatim.prickle\?raw\=true Nominatim.prickle

In [2]:
%run -i "/home/stevengt/Documents/code/whatwhy/whatwhy/main.py"

In [3]:
df